In [1]:
#Pubtatorからからlist取得取得 
import urllib
import pandas as pd

def export(input_path,output_path):
    
    #get pmids
    with open(input_path) as f:
        pmid_ls = f.read().splitlines()
    pmid_ls = sorted(pmid_ls)#sort pmid list
    dic_title ={}#dictionary to store abstracts
    dic_abst = {}#dictionary to store abstracts
    for pmid in pmid_ls:
        dic_title[pmid]="Not found"
        dic_abst[pmid]="Not found"
    i = 0#batch
    while 100*i < len(pmid_ls):#retrieve 100 abstracts in each batch
        html = "https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids="
        pmid_ls_part = pmid_ls[100*i:min(100*(i+1),len(pmid_ls))]
        for pmid in pmid_ls_part:
            html = html+str(pmid)+","
        html = html[:-1]#remove last ","
        #get contents
        response = urllib.request.urlopen(html)
        print("url:", response.geturl())
        content = response.readlines()
        content_str = []#change content datatype to str
        for line in content:
            content_str.append(str(line))
        for text in content_str:
            if "|t|" in text:
                title = text[text.find("|t|")+3:-3]#get after |t| tag and remov \n [:-3]
                dic_title[text[2:text.find("|t|")]]=title
            if "|a|" in text:
                abst = text[text.find("|a|")+3:-3]#get after |a| tag　and remove \n [:-3]
                dic_abst[text[2:text.find("|a|")]]=abst
        i = i+1        
        #merge pmid_ls, abs_ls
    df_index =pd.DataFrame(dic_title.keys(),columns=["PMID"],index=dic_title.keys())
    df_title =pd.DataFrame(dic_title.values(),columns=["title"],index=dic_title.keys())
    df_abst =pd.DataFrame(dic_abst.values(),columns=["abstract"],index=dic_abst.keys())
    df=pd.concat([df_index,df_title,df_abst],axis=1)
    df.set_index("PMID", inplace=True)
    display(df)
    df.to_csv(output_path)

    #retrieve PMID not stored in PTC
    df_none = df[(df["title"] == "Not found")|(df["abstract"] == "Not found")]
   
    print(str(len(pmid_ls)-len(df_none.index))+" titles/abstracts were successfully exported.")
    if df_none.size>0:
        id_none =""
        for id in df_none.index:
            id_none = id_none+id+" " 
        print("Publications of "+id_none+" were not found in PTC. Please add titles and abstracts manually.")
    response.close()

input_path = "../PMID_DM.txt"
output_path = "../DM.csv"
export(input_path,output_path)

url: https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/pubtator?pmids=21824105,21979790,22006544,22138078,22232027,22234928,22276559,22281416,22297054,22310595,22338061,22353395,22382611,22396467,22411919,22422492,22440988,22440995,22456294,22466072,22488512,22519906,22521274,22548941,22553232,22588607,22673502,22712786,22727533,22730470,22753665,22761409,22770818,22795925,22859063,22959138,23063375,23068961,23115224,23117285,23131255,23139978,23142017,23217267,23220882,23237398,23237399,23255310,23258576,23281228,23289822,23312931,23336425,23337144,23345049,23350795,23389397,23391367,23410793,23518842,23587368,23627775,23666582,23672623,23711419,23729552,23756090,23812678,23817340,23829941,23846733,23864222,23874094,23888382,23944865,23945447,23953270,23971019,23981104,24019430,24057287,24090379,24092826,24099071,24151766,24169807,24204764,24254975,24262503,24263644,24283619,24295059,24300630,24341049,24420389,24429126,24429127,24434090,24440167,24446385
url: http

,title,abstract
PMID,,
21824105,Risk factors for cardiovascular events in Japa...,OBJECTIVE: The long-term event monitoring (LEM...
21979790,Efficacy of metformin for prevention of weight...,There is uncertainty with regard to the approp...
22006544,Nonsteroidal antiinflammatory drug intake acco...,OBJECTIVE: To evaluate the interest of the Ass...
22138078,Adapting to the new consensus guidelines for m...,OBJECTIVE: To report our preliminary experienc...
22232027,Insulin glargine or neutral protamine Hagedorn...,OBJECTIVE: To determine the benefit of neutral...
...,...,...
28509500,[Correction of impaired glucose tolerance usin...,The aim of the investigation was comparative s...
29087153,[GLYCEMIC CONTROL IN DIABETES MELLITUS PATIENT...,The management of hyperglycemia in patients wi...
29900709,"Effect of Low (7.5 mg/day), Standard (15 mg/ d...",Objective: To study the effect of different da...


322 titles/abstracts were successfully exported.
